## 11/19
- `mat[uid,iid]`は、評価がめちゃ高い場合のみ1.0になるスパースな行列
- 大きさとしてはlen(uid)*len(iid)分存在。
- consumerのみ、またはshopのみの画像しか存在しないディレクトリを削除する関数を作成。念の為T_Shirt_all2を作成してバックアップ

### get_tripletsについて
- mat.rowの長さ分の乱数を発生->pidとnidが一致することもあるのでは？問題

In [29]:
import scipy.sparse as sp
import numpy as np
mat = sp.lil_matrix((10, 10), dtype=np.int32)

#matの中で正解のペアの部分(mat[uid,pid])のみ1とする。
mat[0,0]=1
mat[0,2]=1
mat[1,4]=1
print(mat)
mat = mat.tocoo()

# matはsparseなので扱うのは値が入っているところのみ
# mat.rowでuidのarray,mat.colでpidのarrayを取得できる
print(mat.row,mat.col)
# mat.shapeはmat全体の長さを返す
print(mat.shape[1])

# mat.rowの長さ分の乱数を発生->pidとnidが一致することもあるのでは？
print(np.random.randint(mat.shape[1], size=len(mat.row)))
print(mat.row,mat.col,np.random.randint(mat.shape[1], size=len(mat.row)))

  (0, 0)	1
  (0, 2)	1
  (1, 4)	1
[0 0 1] [0 2 4]
10
[2 5 3]
[0 0 1] [0 2 4] [6 0 5]


### model.fitの引数とlossについて
- `bpr_triplet_loss`がloss=<font color="red">1.0</font>-{K.sigmoid()}になっている。<br>
- よって、y_trainはnp.ones(len(uid))にしておけば、本来のtriplet lossであるところ？のK.sigmoid()のみが効いてくる。<br>
- ```1.0-{1.0-K.sigmoid()}=K.sigmoid()```<br>
- lossに設定する都合上1.0から引いていると思われる。

In [ ]:
def bpr_triplet_loss(X):

    positive_item_latent, negative_item_latent, user_latent = X

    # BPR loss
    loss = 1.0 - K.sigmoid(
        K.sum(user_latent * positive_item_latent, axis=-1, keepdims=True) -
        K.sum(user_latent * negative_item_latent, axis=-1, keepdims=True))

    return loss

## 11/18
特に何もやっていないので明日やることをまとめる
- 現状`get_triplets(BASE_PATH)`でreturnされるのはconsumer,positive_shop,negative_shopのPATHの組
- PATHの組ではなく画像の組としてreturnした方が学習上都合が良いのでは？
- <b>(済)</b> ~~参考実装は以下のようになっている。`np.random.randint(mat.shape[1], size=len(mat.row))`では`nid`をランダムに抽出している~~
- <font color="red"　>`model.fit`の第二引数が`np.ones(len(uid))`なのはどういうこと？</font>
- triplet lossを利用したfitの場合のラベルに当たる部分を理解する。cf.http://krasserm.github.io/2018/02/07/deep-face-recognition/


In [ ]:
def get_triplets(mat):
    return mat.row, mat.col, np.random.randint(mat.shape[1], size=len(mat.row))

In [ ]:
for epoch in range(num_epochs):

    print('Epoch %s' % epoch)

    # Sample triplets from the training data
    uid, pid, nid = get_triplets(train)

    X = {
        'user_input': uid,
        'positive_item_input': pid,
        'negative_item_input': nid
    }

    model.fit(X,
              np.ones(len(uid)),　# これ何？
              batch_size=64,
              nb_epoch=1,
              verbose=0,
              shuffle=True)

    print('AUC %s' % metrics.full_auc(model, test))

## 11/17
### やったこと
- trainとtestの分割を以下のように定義

In [1]:
def sample_train_ids(ids):
    np.random.seed(seed=32)
    train_ids = np.random.choice(ids,int(0.75*len(ids)))
    return train_ids

In [ ]:
BASE_PATH = './T_Shirt_all/'
ids = sorted([x for x in os.listdir(BASE_PATH)])

train_ids = sample_train_ids(ids)
#idsの中でtrain_idsに含まれていないxをtest_idsとする
test_ids = [x for x in ids if x not in train_ids] 

len(ids):6377<br>
train:4782<br>
test:2999 <br>
という謎の結果になってしまった。
### やること
- 上の現象をチェックする。

## 11/16
### やったこと
- TOPSとCLOTHINGの結合->T_Shirt_allへ
- 関数の作成(get_triplet)
### やること
- <b>(済)</b>　~~T_Shirt_all/trainとT_Shirt_all/testに分割~~ -> 単純にidをランダムで75%取り出す処理に
- <b>(済)</b>　~~分割の仕方はランダムで行う？~~
    - ```train_ids=ids[np.random.choice(len(os.listdir),0.75)]``` 的な感じで75%を取り出せるかな？